# GOES-R DataJam: Setup and Data Access

## Environment Setup

### GitHub


### Creating a Jupyter Notebook

### Install Packages

In [ ]:
# install packages using pip

In [ ]:
# import packages
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
import boto3
import fsspec

## Data Access

In [ ]:
# set up access to S3 bucket
s3 = boto3.client("s3")
paginator = s3.get_paginator("list_objects_v2")
page_iterator = paginator.paginate(Bucket="noaa-goes16", Prefix="ABI-L2-CMIPC/2021/001/18/") 

S3_HEADER = "s3://noaa-goes16/"
files_mapper = [S3_HEADER + f["Key"] for page in page_iterator for f in page["Contents"]]

In [ ]:
files_mapper[0]

In [ ]:
files_mapper[1]

In [ ]:
# open data file
url_blue = files_mapper[1]
ds_blue = xr.open_dataset(url_blue)
# MAY NEED TO USE FSSPEC.OPEN()
ds_blue

### What is a CMI file?

### ABI Imagery Bands

### Plotting an Image

In [ ]:
# plot a simple image with one band
ds_blue["CMI"].plot()

In [ ]:
# inspect other channels

# red
url_red = get_url_for_prefix("ABI-L2-CMIPC/2021/001/18/OR_ABI-L2-CMIPC-M6C02")
ds_red = xr.open_dataset(url_red) # MAY NEED FSSPEC

# vegetation ("veggie") AKA green
url_veggie = get_url_for_prefix("ABI-L2-CMIPC/2021/001/18/OR_ABI-L2-CMIPC-M6C03")
ds_veggie = xr.open_dataset(url_veggie) # MAY NEED FSSPEC

In [ ]:
display(ds_blue["CMI"].values.shape)
display(ds_red["CMI"].values.shape)
display(ds_veggies["CMI"].values.shape)

In [ ]:
# resample red 
red_vals = ds_red["CMI"].values
shape = ds_blue.values.shape

sh = shape[0], red_vals.shape[0]//shape[0], shape[1], red_vals.shape[1]//shape[1]
ds_red_reshape = red_vals.reshape(sh).mean(-1).mean(1)

In [ ]:
ds_red_reshape.shape

In [ ]:
# plot with all colors
geocolor = np.stack(ds_red_reshape, ds_veggie, ds_blue, axis = 2)

fig = plt.figure(figsize=(10,10), dpi = 200)
img = plt.imshow(geocolor)
plt.title("GeoColor Image")
plt.show()

### Plotting an Animation